**Find the smallest sphere that encloses a set of** $k$ **points** $p_i \in \mathbb{R}^n$. 


The problem boils down to determine the sphere center $c_0\in \mathbb{R}^n$ and its radius $r_0\geq 0$, i.e.


\begin{equation}
  \begin{aligned}
\min \max_{i=1,\dots,k} \| p_i - p_0\|_2 \\
  \end{aligned}
\end{equation}

The maximum in the objective function can be easily, i.e.

\begin{equation}
  \begin{aligned}
\min r_0 & & &\\
s.t.& & &\\
& r_i \geq \| p_i - p_0\|_2 ,& \quad & i=1,\ldots,k\\
\end{aligned}
\end{equation}

------------------------------

For sake of simplicity we will consider

* $n=2$,
* no restriction on the sphere center location.

Our input data will be generate randomly around the origin with a standard normal distibution.

Here it is:

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def plot_points(p, p0=[], r0=0.):
    n,k= len(p0), len(p)
    
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.plot(  p[:,0], p[:,1], 'b*')
    
    if len(p0)>0:
        ax.plot(  p0[0], p0[1], 'r.')
        ax.add_patch( mpatches.Circle( p0,  r0 ,  fc="w", ec="r", lw=1.5) )
        
    plt.show()

n = 2
k = 100

p=  np.random.normal(size=(k,n))


plot_points(p)


NameError: name 'np' is not defined

The SOCP formulation reads

\begin{equation}
  \begin{aligned}
\min r_0 & & &\\
s.t.& & &\\
& (r_0,p_i - p_0) \in Q^{(n+1)}, & \quad & i=1,\ldots,k.
\end{aligned}
\end{equation}

The `fusion` implementatin starts defining the optimization model:

Before defining the constraints, we note that we can write


\begin{equation}
r = \left(r_0,\ldots,r_0\right)^T \in \mathbb{R}^k
\end{equation}

and 

\begin{equation}
P = \left(p_0,\ldots,p_k\right)^T \in \mathbb{R}^{k\times n}.
\end{equation}

so that 

\begin{equation}
(r_0,p_i - p_0) \in Q^{(n+1)},  \quad  i=1,\ldots,k.
\end{equation}

can be compactly expressed as 

\begin{equation}
(r,P) \in \Pi Q^{(n+1)}.
\end{equation}

Now we are ready for a compact implementation in `Fusion`:

In [2]:
from mosek.fusion import *

M = Model("minimal sphere enclosing a set of points")

r0 = M.variable(1         , Domain.greaterThan(0.))
p0 = M.variable(NDSet(1,n), Domain.unbounded())

M.constraint(\
    Expr.hstack(\
                Variable.repeat(r0,k),
                Expr.sub( Variable.repeat(p0,k), DenseMatrix(p) )\
               ),\
        Domain.inQCone())

M.setLogHandler(open('logp','wt'))

M.objective(ObjectiveSense.Minimize, r0)

M.solve()

print "r0^* = ", r0.level()[0]
print "p0^* = ", p0.level()

NameError: name 'p' is not defined

In [3]:
plot_points(p,p0.level(),r0.level()[0])

NameError: name 'p' is not defined

= Dual Formulation 

Let's derive the dual:

\begin{aligned}
    \max & \sum_i b_i^T y_i \\
    & \sum_i y_i = c \\
    & y_i \in \mathcal{Q}^{n+1}
\end{aligned}


where $c=\left(1, 0_n\right)^T$ and $b_i=(0,p_i)$. Introducing 

\begin{equation}
 Y = \left[ y_0, \ldots,y_k  \right], B=\left[ b_1,\ldots,b_k\right]
\end{equation}


\begin{aligned}
    \max & \left< B, Y \right> \\
    & Y \mathbb{1}_k = c \\
    & Y_{:,i} \in \mathcal{Q}^{n+1}
\end{aligned}



In [4]:
Md = Model()

y= Md.variable(NDSet(k,n+1), Domain.inQCone(k,n+1))
Md.constraint( Expr.mul( [1.0 for i in range(k)], y), Domain.equalsTo( [1.] + [0. for i in range(n)]) )
    
Md.objective(ObjectiveSense.Maximize, Expr.dot( DenseMatrix(p.tolist()), y.slice([0,1],[k,n+1])) )
 
Md.setLogHandler(open('logd','wt'))

Md.solve()


NameError: name 'p' is not defined

Let's take a closer look to the solver output:

In [5]:
!tail  log*

==> logd <==
7   1.9e-02  2.8e-04  1.9e-04  1.05e+00   2.593207070e+00   2.592398084e+00   1.9e-04  0.00  
8   1.3e-03  1.9e-05  1.3e-05  1.00e+00   2.589033354e+00   2.588984398e+00   1.3e-05  0.01  
9   7.4e-05  1.1e-06  7.4e-07  1.00e+00   2.588805277e+00   2.588802489e+00   7.4e-07  0.01  
10  6.3e-06  9.3e-08  6.4e-08  1.00e+00   2.588793760e+00   2.588793522e+00   6.4e-08  0.01  
11  8.5e-07  1.3e-08  8.5e-09  1.00e+00   2.588793113e+00   2.588793081e+00   8.6e-09  0.01  
12  7.3e-08  1.2e-09  9.2e-09  1.00e+00   2.588793042e+00   2.588793061e+00   8.0e-10  0.01  
13  2.0e-08  7.6e-11  7.0e-09  1.00e+00   2.588793080e+00   2.588793061e+00   5.2e-11  0.01  
Interior-point optimizer terminated. Time: 0.01. 

Optimizer terminated. Time: 0.01    

==> logp <==
5   8.5e-03  5.3e-03  1.1e-02  1.54e+00   2.703780523e+00   2.716071112e+00   5.3e-03  0.03  
6   3.3e-03  2.0e-03  4.1e-03  1.10e+00   2.644672935e+00   2.648460754e+00   2.0e-03  0.03  
7   3.1e-04  1.9e-04  3.8e-04  1.05e+00

In [6]:
!grep Scalar log*

logd:  Scalar variables       : 300             
logd:Optimizer  - Scalar variables       : 300               conic                  : 300             
logp:  Scalar variables       : 303             
logp:Optimizer  - Scalar variables       : 300               conic                  : 300             


In [7]:
!grep Cones log*

logd:  Cones                  : 100             
logd:Optimizer  - Cones                  : 100
logp:  Cones                  : 100             
logp:Optimizer  - Cones                  : 100


In [9]:
!grep Constraints log*


logd:  Constraints            : 3               
logd:Optimizer  - Constraints            : 3
logp:  Constraints            : 300             
logp:Optimizer  - Constraints            : 297


Why is that? `Fusion` perform the following transformation:

\begin{equation}
Ax - b \in \mathcal{K} \rightarrow \quad \left\lbrace\begin{array}{ll} y = Ax-b\\ y\in \mathcal{K} \end{array}\right.
\end{equation}

The reason is to ensure each variable to belong only in cone (in the `optimizer API` this must be done by the user).

But this is not the reason: 

* **MOSEK** uses a primal-dual algorithm that will solve both formulation at the same time,
* therefore internally there is no difference;


**So why is the dual formulation so much faster?**

The answer comes from a closer inspection to the solver output, in particular the reported `flops`:


The important point is the the flop and the dense dimensions

In [10]:
!grep flop log*

logd:Factor     - dense dim.             : 0                 flops                  : 2.63e+03        
logp:Factor     - dense dim.             : 2                 flops                  : 4.21e+06        


In [11]:
4.21e+06/2.63e+03

1600.7604562737642

When moving from thoery to practical implementation, **MOSEK** is somehow biased towards the primal, i.e. the formulation we input:

* some steps, most notably fatorizations, are order dependent,
* it is important to not be mislead by the problem dimension looking only at the number of variables, constraints and cones
* if performance are a concern, both primal and dual should be tried.

Hopefully we will include an automatic dualizer tool in the future release. But it will be still a heuristic.

